## Support Tensor machine

Given a set of training samples $\{X_i, y_i\}$, $i= 1,···, m$, where $X_i$ is the data point in order-2 tensor space, $X_i \in R^{n_1} \times  R^{n_2}$ and $y_i \in \{−1,1\}$ is the label associated with $X_i$. Find a tensor classifier $f(X) =u^TXv+b$ such that the two classes can be separated with maximum margin.


**Algorithm**: Train Support Tensor Machine

**Input**: $\{X_i, y_i\}$, $i= 1,···, m$. $X_i \in R^{n_1} \times  R^{n_2}$ and $y_i \in \{−1,1\}$ is the label associated with $X_i$. $maxIter$ is an integer greater than 0. $C_1$, and $C_2$ are regularization paramaters, real values greater than 0. And $tol$, is a tolerance for convergence.

**Output**: $u_{opt} \in R^{n_1}$, $v_{opt} \in R^{n_2}$ and $b_{opt} \in R$ are parameters for decision function $f(X) =u^T_{opt}Xv_{opt}+b_{opt}$.

1. **Initialize** $u_{opt} = (1,...,1)^T \in R^{n_1}$ and $v_{opt} = (1,...,1)^T \in R^{n_2}$
2. **for** $k=1$ **to** $maxIter$ **do**:
3. ---- $x^{u}_{i} = X^{T}_{i}u_{opt}$
4. ---- $v^{k}_{opt}, b^{k}_{opt} = $ **LinearSVM**$(\{x^{u}_{i},y_i\}, C_1)$
5. ---- $x^{v}_{i} = X_{i}v_{opt}$
6. ---- $u^{k}_{opt}, b^{k}_{opt} = $ **LinearSVM**$(\{x^{v}_{i},y_i\}, C_2)$
7. ---- **if** $|u^{k}_{opt} - u_{opt}| < tol$ **and** $|v^{k}_{opt} - v_{opt}| < tol$: 
8. -------- **break**
9. ---- **end if**
10. ---- $u_{opt} = u^{k}_{opt}$
11. ---- $v_{opt} = v^{k}_{opt}$
12. **end for**
13. **return** $u_{opt}$, $v_{opt}$, $b_{opt}$

In [1]:
!pip3 install mne
!pip3 install wget

     |████████████████████████████████| 7.0MB 4.3MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=2cf3feaab5906ec925aa3368e06546a8c274b34842b92c98ffebd4d959f7acb6
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import mne
from scipy import stats
from mne.stats import bonferroni_correction, fdr_correction
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
import pickle
import wget
import pandas as pd
import zipfile
import os
import shutil
import numpy as np
import time
from numpy.fft import rfft
from scipy import signal
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import combinations
from sklearn.model_selection import ShuffleSplit, cross_val_score

In [3]:
"""Module for classification function."""
import numpy as np

from scipy import stats

from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils.extmath import softmax
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from joblib import Parallel, delayed

#PARAMETERS Linear SVM
#C: float, default=1.0
#Regularization parameter. The strength of the regularization 
#is inversely proportional to C. Must be strictly positive.

#penalty: {‘l1’, ‘l2’}, default=’l2’
#Specifies the norm used in the penalization. 
#The ‘l2’ penalty is the standard used in SVC. 
#The ‘l1’ leads to coef_ vectors that are sparse.

#loss: {‘hinge’, ‘squared_hinge’}, default=’squared_hinge’
#Specifies the loss function. ‘hinge’ is the standard SVM 
#loss (used e.g. by the SVC class) while ‘squared_hinge’ is 
#the square of the hinge loss. The combination of penalty='l1' 
#and loss='hinge' is not supported.

#tol: float, default=1e-4
#Tolerance for stopping criteria.

#dual: bool, default=True
#Select the algorithm to either solve the dual or primal optimization problem. 
#Prefer dual=False when n_samples > n_features.


class STM(BaseEstimator, ClassifierMixin, TransformerMixin):

    """Classification by Support Tensor Machine.
    """

    def __init__(self, C1=1.0, C2=1.0, maxIter=30, tolSTM=1e-4, penalty = 'l2', dual = True, tol=1e-4,loss = 'squared_hinge', maxIterSVM=100000, n_jobs=1):
        """Init."""
        # store params for cloning purpose
        self.C1 = C1
        self.C2 = C2
        self.dual = dual
        self.penalty = penalty
        self.tolSTM = tolSTM
        self.tol = tol
        self.loss = loss
        self.maxIter = maxIter
        self.maxIterSVM = maxIterSVM
        self.n_jobs = n_jobs

    def fit(self, X, y, sample_weight=None):
        """Fit (estimates) the u and v.
        Parameters
        ----------
        X : ndarray, shape (n_trials, n_channels, n_samples)
            ndarray of matrices.
        y : ndarray shape (n_trials, 1)
            labels corresponding to each trial.
        sample_weight : None | ndarray shape (n_trials, 1)
            the weights of each sample. if None, each sample is treated with
            equal weights.
        Returns
        -------
        self : STM instance
            The STM instance.
        """
        self.classes_ = sorted([round(val) for val in np.unique(y)])
        self.dictclassestoval = dict(zip(self.classes_,[-1,+1]))
        self.dictvaltoclasses = dict(zip([-1,+1],self.classes_))
        y = np.array([self.dictclassestoval[round(val)] for val in y])

        if sample_weight is None:
            sample_weight = np.ones(X.shape[0])

        #if self.n_jobs == 1:
        #STM: Support Tensor Machine
        n_channels = X.shape[1]
        n_trials = X.shape[0]
        n_samples = X.shape[2]
        u = np.ones(n_channels)
        v = np.ones(n_samples)
        #print(u.shape)

        totaliterations = self.maxIter
        for i in range(totaliterations):
          print('%d / %d'%(i+1,totaliterations))
          # First SVM on time
          X_t = []
          for i in range(n_trials):
            x = X[i,:,:].T.dot(u)
            X_t.append(x)
          X_t = np.stack(X_t,axis=0)
          #print('X_t.shape, y.shape', X_t.shape, y.shape)

          clf1 = LinearSVC(C=self.C1, dual=self.dual, penalty=self.penalty, loss=self.loss, tol=self.tol, max_iter=self.maxIterSVM)
          clf1.fit(X_t, y)
          coef_ = clf1.coef_
          intercept_ = clf1.intercept_
          n_iter_ = clf1.n_iter_
          #print('n_iter', n_iter_)

          v_next = coef_.copy()
          v_intercept = intercept_
          deltav = np.linalg.norm(v - v_next)
          v = v_next.copy()
          v_iter = n_iter_
          #print('v.shape',v.shape)

          # Second SVM on channel
          X_c = []
          for i in range(n_trials):
            x = X[i,:,:].dot(v.T)[:,0]
            X_c.append(x)
          X_c = np.stack(X_c,axis=0)
          #print('X_c.shape, y.shape', X_c.shape, y.shape)

          clf2 = LinearSVC(C=self.C2, dual=self.dual, penalty=self.penalty, loss=self.loss, tol=self.tol, max_iter=self.maxIterSVM)
          clf2.fit(X_c, y)
          coef_ = clf2.coef_
          intercept_ = clf2.intercept_
          n_iter_ = clf2.n_iter_
          #print('n_iter', n_iter_)

          u_next = coef_.copy()
          u_intercept = intercept_
          deltau = np.linalg.norm(u - u_next)
          u = u_next.copy()
          u_iter = n_iter_
          #print('u.shape',u.shape)
          u = np.squeeze(u)
          
          self.deltau_ = deltau
          self.deltav_ = deltav
          print("deltau, deltav", deltau, deltav)
          if self.deltau_ < self.tolSTM and self.deltav_ < self.tolSTM:
            break
        self.u_opt_ = u_next.copy()
        self.v_opt_ = v_next.copy()
        self.intercept_ = u_intercept[0]

        X_pred = []
        for i in range(X.shape[0]):
          x = X[i,:,:]
          x_v = x.dot(self.v_opt_.T)
          x_u = float(self.u_opt_.dot(x_v))
          f = x_u + self.intercept_
          X_pred.append(f)
        X_pred = np.array(X_pred).reshape(X.shape[0],1)
        clf_log = LogisticRegression(random_state=0).fit(X_pred, y)
        self.LogisticRegression = clf_log
        
        return self

    def predict(self, X):
        """get the predictions.
        Parameters
        ----------
        X : ndarray, shape (n_trials, n_channels, n_channels)
            ndarray of SPD matrices.
        Returns
        -------
        pred : ndarray of int, shape (n_trials, 1)
            the prediction for each trials according to the closest centroid.
        """
        pred = []
        for i in range(X.shape[0]):
          x = X[i,:,:]
          x_v = x.dot(self.v_opt_.T)
          x_u = float(self.u_opt_.dot(x_v))
          f = round(np.sign(x_u + self.intercept_))
          pred.append(self.dictvaltoclasses[f])      
        return np.array(pred)

    def decision_function(self, X):
        """get the predictions.
        Parameters
        ----------
        X : ndarray, shape (n_trials, n_channels, n_channels)
            ndarray of SPD matrices.
        Returns
        -------
        pred : ndarray of int, shape (n_trials, 1)
            the prediction for each trials according to the closest centroid.
        """
        pred = []
        for i in range(X.shape[0]):
          x = X[i,:,:]
          x_v = x.dot(self.v_opt_.T)
          x_u = float(self.u_opt_.dot(x_v))
          f = x_u + self.intercept_
          pred.append(f)      
        return np.array(pred)

    def fit_predict(self, X, y):
        """Fit and predict in one function."""
        self.fit(X, y)
        return self.predict(X)

    def predict_proba(self, X):
        X_pred = []
        for i in range(X.shape[0]):
          x = X[i,:,:]
          x_v = x.dot(self.v_opt_.T)
          x_u = float(self.u_opt_.dot(x_v))
          f = x_u + self.intercept_
          X_pred.append(f)
        X_pred = np.array(X_pred).reshape(X.shape[0],1)
        y_proba = self.LogisticRegression.predict_proba(X_pred)
        return y_proba

In [4]:
from scipy.signal import butter, lfilter, filtfilt

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [5]:
#Retrived from: http://www.jtrive.com/the-empirical-bootstrap-for-confidence-intervals-in-python.html
def bootstrap(data, n=1000, func=np.mean):
    """
    Generate `n` bootstrap samples, evaluating `func`
    at each resampling. `bootstrap` returns a function,
    which can be called to obtain confidence intervals
    of interest.
    """
    simulations = list()
    sample_size = len(data)
    xbar_init = np.mean(data)
    for c in range(n):
        itersample = np.random.choice(data, size=sample_size, replace=True)
        simulations.append(func(itersample))
    simulations.sort()
    def ci(p):
        """
        Return 2-sided symmetric confidence interval specified
        by p.
        """
        u_pval = (1+p)/2.
        l_pval = (1-u_pval)
        l_indx = int(np.floor(n*l_pval))
        u_indx = int(np.floor(n*u_pval))
        return(simulations[l_indx],simulations[u_indx])
    return(ci)

In [6]:
def CARFiltering(df, channels_interest):
  dictData = df.to_dict(orient='list')
  for i in range(len(df)):
    meanVal = float(np.mean([float(dictData[col][i]) for col in channels_interest]))
    for col in channels_interest:
      dictData[col][i] = dictData[col][i] - meanVal
  df = pd.DataFrame(dictData)
  return df.copy()

In [7]:
def getDFData(URL, User):
  #Data Description: https://lampx.tugraz.at/~bci/database/001-2019/dataset_description_v1-1.pdf
  #Offline: Type run and event codes
  dictTypeRun =  {'Run 1':'EyeMovements',
                  'Run 2':'Rest',
                  'Run 3':'AttemptedMovement',
                  'Run 4':'AttemptedMovement',
                  'Run 5':'AttemptedMovement',
                  'Run 6':'AttemptedMovement',
                  'Run 7':'AttemptedMovement',
                  'Run 8':'EyeMovements',
                  'Run 9':'Rest',
                  'Run 10':'AttemptedMovement',
                  'Run 11':'AttemptedMovement',
                  'Run 12':'AttemptedMovement',
                  'Run 13':'AttemptedMovement',
                  'Run 14':'EyeMovements',
                  'Run 15':'Rest'
                 }
  dictEvents = dict(TrialStart = 0x300,
                    Beep = 0x311,
                    FixationCross = 0x312,
                    SupinationClassCue = 0x308,
                    PronationClassCue = 0x309,
                    HandOpenClassCue = 0x30B,
                    PalmarGraspClassCue = 0x39D,
                    LateralGraspClassCue = 0x39E)
  dictEvents = dict(zip([str(val) for val in list(dictEvents.values())],list(dictEvents.keys())))
  dictColNames = dict(zip(list(dictEvents.keys()), list(range(len(list(dictEvents.keys()))))))
  
  #Unzip on User folder
  try:
    shutil.rmtree(User)
    os.mkdir(User)
  except:
    os.mkdir(User)  
  
  
  if not os.path.exists(User+'.zip'):
    # Download file
    print('Downloading: ',User,' from ',URL+User+'.zip')
    filename = wget.download(URL+User+'.zip')
  else:
    filename = User+'.zip'


  
  with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(User)
  RunFiles = [os.path.join(User,f) for f in os.listdir(User) if os.path.isfile(os.path.join(User, f)) and f.lower().endswith('.gdf')]
  
  #Prepare DataFrame
  listDFRaw = []
  typeRun = []
  numRun = []
  samplingFrequencyList = []


  
  raw_all = mne.concatenate_raws([mne.io.read_raw_gdf(f, preload=True) for f in RunFiles])
  for run in dictTypeRun:
    rfile = None
    for runfile in RunFiles:
      if run in runfile:
        rfile = runfile
        break
    if rfile is None:
      continue
    
    raw = mne.io.read_raw_gdf(rfile)
    
    samplingFrequencyList.append(raw.info['sfreq'])
    ch_names = raw.info['ch_names']
    dfData = pd.DataFrame(data=raw.get_data().T,columns=ch_names)
    dfData = dfData.dropna(how='all')
    dfData.fillna(method='bfill',inplace=True)
    dfData = dfData.reset_index()
    dfData = dfData[ch_names]

    events, dictEventsRun = mne.events_from_annotations(raw)
    dictEventsRun = dict(zip([val for val in list(dictEventsRun.values())],list(dictEventsRun.keys())))
    sampleTime, eventNum = list(events[:,0]), list(events[:,2])
    listEventsPerColumn = [[0]*len(dfData)]*len(dictColNames)
    listEventsPerColumn = np.array(listEventsPerColumn)
    for s, e in zip(sampleTime, eventNum):
      if dictEventsRun[e] in dictColNames:
        listEventsPerColumn[dictColNames[dictEventsRun[e]],s] = 1
    dfEvents = pd.DataFrame(data=listEventsPerColumn.T,columns=[dictEvents[val] for val in list(dictColNames.keys())])
    dfRaw = pd.concat([dfEvents.copy(),dfData.copy()],axis=1,ignore_index=True)
    dfRaw.columns = [dictEvents[val] for val in list(dictColNames.keys())] + ch_names
    

    listDFRaw.append(dfRaw.copy())
    typeRun += [dictTypeRun[run]]*len(dfData)
    numRun += [run]*len(dfData)

  shutil.rmtree(User) 

  #Build DataFrame and return
  df = pd.concat(listDFRaw, ignore_index=True)
  df['TypeRun'] = typeRun
  df['Run'] = numRun
  df = df[['Run','TypeRun']+[dictEvents[val] for val in list(dictColNames.keys())]+ch_names]
  return df, list(set(samplingFrequencyList)), raw_all

## 1. Setting

In [8]:
dictEvents = dict(TrialStart = 0x300,
                  Beep = 0x311,
                  FixationCross = 0x312,
                  SupinationClassCue = 0x308,
                  PronationClassCue = 0x309,
                  HandOpenClassCue = 0x30B,
                  PalmarGraspClassCue = 0x39D,
                  LateralGraspClassCue = 0x39E)
dictEvents

{'Beep': 785,
 'FixationCross': 786,
 'HandOpenClassCue': 779,
 'LateralGraspClassCue': 926,
 'PalmarGraspClassCue': 925,
 'PronationClassCue': 777,
 'SupinationClassCue': 776,
 'TrialStart': 768}

In [9]:
typeRun = 'AttemptedMovement'
typeRest = 'Rest'
eeg_channels = ['AFz', 'F3', 'F1', 'Fz', 'F2', 'F4', 'FFC5h', 'FFC3h', 'FFC1h', 'FFC2h', 
                'FFC4h', 'FFC6h', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FCC5h', 
                'FCC3h', 'FCC1h', 'FCC2h', 'FCC4h', 'FCC6h', 'C5', 'C3', 'C1', 'Cz', 'C2', 
                'C4', 'C6', 'CCP5h', 'CCP3h', 'CCP1h', 'CCP2h', 'CCP4h', 'CCP6h', 'CP5', 
                'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'CPP5h', 'CPP3h', 'CPP1h', 'CPP2h', 
                'CPP4h', 'CPP6h', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'PPO1h', 'PPO2h', 'POz']

movementTypes = ['SupinationClassCue','PronationClassCue','HandOpenClassCue', 'PalmarGraspClassCue','LateralGraspClassCue']
colors = ['red','green', 'blue', 'indigo','yellow','gray']
lowcut = 0.3
highcut = 3
butter_order = 4
tmin = -2.0
tmax = 3.0
bootstrapiterations = 100
confidenceinterval = .95
channel = 'C2'
Users = ['P01','P02','P03','P04','P05','P06','P07','P08','P09','P10']

In [10]:
df, sfreq, raw_mne = getDFData(URL='http://bnci-horizon-2020.eu/database/data-sets/001-2019/', User=Users[0])
ch_types = {}
for ch in raw_mne.ch_names:
  if not ch in ['eog-l','eog-m','eog-r']:
    ch_types[ch] = 'eeg'
  else:
    ch_types[ch] = 'eog'
raw_mne.set_channel_types(ch_types)
montage = make_standard_montage('standard_1005')
raw_mne.set_montage(montage)

raw_mne, _ = mne.set_eeg_reference(raw_mne, 'average',copy=True)

Downloading:  P01  from  http://bnci-horizon-2020.eu/database/data-sets/001-2019/P01.zip
Extracting EDF parameters from /content/P01/P01 Run 4.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...


/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead



Reading 0 ... 76543  =      0.000 ...   298.996 secs...
Extracting EDF parameters from /content/P01/P01 Run 13.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77311  =      0.000 ...   301.996 secs...
Extracting EDF parameters from /content/P01/P01 Run 15.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 23295  =      0.000 ...    90.996 secs...
Extracting EDF parameters from /content/P01/P01 Run 11.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77055  =      0.000 ...   300.996 secs...


/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead

/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead

/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead



Extracting EDF parameters from /content/P01/P01 Run 12.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77823  =      0.000 ...   303.996 secs...
Extracting EDF parameters from /content/P01/P01 Run 10.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77823  =      0.000 ...   303.996 secs...


...

Extracting EDF parameters from /content/P01/P01 Run 13.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['33536', '33544', '33545', '33547', '33554', '33693', '33694', '768', '776', '777', '779', '785', '786', '925', '926']


/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead



Extracting EDF parameters from /content/P01/P01 Run 14.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['33536', '768', '785']
Extracting EDF parameters from /content/P01/P01 Run 15.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['33536', '33554', '768', '785', '786']


/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead

/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead



EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.


In [11]:
# Transforming events
picks = pick_types(raw_mne.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
events_before, dictEventsRun = mne.events_from_annotations(raw_mne)
dictEventsRun = dict(zip( list(dictEventsRun.values()) , [int(val) for val in list(dictEventsRun.keys())] ))
events = events_before.copy()
for i in range(events_before.shape[0]):
  events[i,2]=dictEventsRun[events[i,2]]

Used Annotations descriptions: ['33536', '33544', '33545', '33547', '33554', '33693', '33694', '768', '776', '777', '779', '785', '786', '925', '926']


In [12]:
event_id = dictEvents
tmin=-2.0
tmax = 3.0
epochs = Epochs(raw_mne, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True, event_repeated='merge')

Not setting metadata
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
729 matching events found
No baseline correction applied
0 projection items activated
Loading data for 729 events and 1281 original time points ...
0 bad epochs dropped


In [13]:
epochs

Number of events,729
Events,Beep: 3Beep/FixationCross/TrialStart: 363Beep/TrialStart: 3HandOpenClassCue: 72LateralGraspClassCue: 72PalmarGraspClassCue: 72PronationClassCue: 72SupinationClassCue: 72
Time range,-2.000 – 3.000 sec
Baseline,off


In [14]:
sfreq = sfreq[0]

In [15]:
df

,Run,TypeRun,TrialStart,Beep,FixationCross,SupinationClassCue,PronationClassCue,HandOpenClassCue,PalmarGraspClassCue,LateralGraspClassCue,AFz,F3,F1,Fz,F2,F4,FFC5h,FFC3h,FFC1h,FFC2h,FFC4h,FFC6h,FC5,FC3,FC1,FCz,FC2,FC4,FC6,FCC5h,FCC3h,FCC1h,FCC2h,FCC4h,FCC6h,C5,C3,C1,Cz,C2,C4,C6,CCP5h,CCP3h,CCP1h,CCP2h,CCP4h,CCP6h,CP5,CP3,CP1,CPz,CP2,CP4,CP6,CPP5h,CPP3h,CPP1h,CPP2h,CPP4h,CPP6h,P5,P3,P1,Pz,P2,P4,P6,PPO1h,PPO2h,POz,eog-l,eog-m,eog-r
0,Run 1,EyeMovements,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Run 1,EyeMovements,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Run 1,EyeMovements,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Run 1,EyeMovements,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Run 1,EyeMovements,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880820,Run 15,Rest,0,0,0,0,0,0,0,0,174.301636,1500.522217,22.518784,-81.201134,128.299026,-77.132866,1154.862427,-85.674629,79.631111,-12.415437,295.127686,170.730377,359.860626,169.677612,175.660080,-6

###0. Which is a filtered version of Data?

In [16]:
maincols = ['Run','TypeRun','TrialStart','Beep','FixationCross']

eeg_channels_of_interest = ['AFz', 'F3', 'F1', 'Fz', 'F2', 'F4', 'FFC5h', 'FFC3h', 'FFC1h', 'FFC2h', 
                'FFC4h', 'FFC6h', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FCC5h', 
                'FCC3h', 'FCC1h', 'FCC2h', 'FCC4h', 'FCC6h', 'C5', 'C3', 'C1', 'Cz', 'C2', 
                'C4', 'C6', 'CCP5h', 'CCP3h', 'CCP1h', 'CCP2h', 'CCP4h', 'CCP6h', 'CP5', 
                'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'CPP5h', 'CPP3h', 'CPP1h', 'CPP2h', 
                'CPP4h', 'CPP6h', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'PPO1h', 'PPO2h', 'POz']

movementTypes_of_interest = ['SupinationClassCue','PronationClassCue','HandOpenClassCue', 'PalmarGraspClassCue','LateralGraspClassCue']

In [17]:
dfFiltered = df[maincols+movementTypes_of_interest+eeg_channels_of_interest]
dfFiltered = CARFiltering(dfFiltered, eeg_channels_of_interest)
dfFiltered.head(5)

,Run,TypeRun,TrialStart,Beep,FixationCross,SupinationClassCue,PronationClassCue,HandOpenClassCue,PalmarGraspClassCue,LateralGraspClassCue,AFz,F3,F1,Fz,F2,F4,FFC5h,FFC3h,FFC1h,FFC2h,FFC4h,FFC6h,FC5,FC3,FC1,FCz,FC2,FC4,FC6,FCC5h,FCC3h,FCC1h,FCC2h,FCC4h,FCC6h,C5,C3,C1,Cz,C2,C4,C6,CCP5h,CCP3h,CCP1h,CCP2h,CCP4h,CCP6h,CP5,CP3,CP1,CPz,CP2,CP4,CP6,CPP5h,CPP3h,CPP1h,CPP2h,CPP4h,CPP6h,P5,P3,P1,Pz,P2,P4,P6,PPO1h,PPO2h,POz
0,Run 1,EyeMovements,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Run 1,EyeMovements,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Run 1,EyeMovements,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Run 1,EyeMovements,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Run 1,EyeMovements,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### **1. ¿How many "Trial Start" events do Runs of type Rest are in the data and what is the temporal distance between cosecutive "Trial Start" events?**

In [18]:
### Analysis
# Filter by TypeRun equal to Rest
dfRest = dfFiltered[dfFiltered['TypeRun']=='Rest']
dfRest.reset_index(inplace=True)
#Signal rest
signalRest = dfRest['TrialStart'].values.tolist()
#Retrieve indexes where signal equals 1
indexesOnes = [i for i,val in enumerate(signalRest) if val == 1]
timesOnes = [val/(sfreq) for val in indexesOnes]
consecutiveOnes = [timesOnes[i+1]-timesOnes[i] for i,val in enumerate(timesOnes) if i+1 < len(timesOnes)]

### Report
print('Report on REST:')
print('Total samples = %d'%(len(signalRest)))
print('Sampling frequency (Hz) = %d'%(sfreq))
print('Signal duration (s) = %.2f'%(len(signalRest)/sfreq))
print('Signal duration (min) = %.2f'%(len(signalRest)/(60.0*sfreq)))
print('Total trial start events = '+str(len(indexesOnes)))
print('Indexes (sample) = '+str(indexesOnes))
print('Times (s) = '+str(timesOnes))
print('Duration between consecutive ones (s) = '+str(consecutiveOnes))

Report on REST:
Total samples = 69129
Sampling frequency (Hz) = 256
Signal duration (s) = 270.04
Signal duration (min) = 4.50
Total trial start events = 3
Indexes (sample) = [2560, 25603, 48646]
Times (s) = [10.0, 100.01171875, 190.0234375]
Duration between consecutive ones (s) = [90.01171875, 90.01171875]


#### **2. What are 72 epochs associated with REST?**

R:/ From every "Trial Start" event we can slide window of duration: tmax - tmin.

In [19]:
#Estimating start and end for every epoch
totalEpochs = 72
epochsPerOnes = round(totalEpochs/len(indexesOnes))
starts = []
ends = []
startstimes = []
endstimes = []
offset = 0.5*sfreq # One half second
totalSamples = round(1 + ((tmax - tmin)*sfreq))
steps = round(0.5*totalSamples) #10% of advancing (90% traslaping)

for starIndexOnes in indexesOnes:
  for i in range(epochsPerOnes):
    start = round(starIndexOnes+offset+(i*steps))
    starts.append(start)
    ends.append(start+totalSamples)

#Transforming from sample to time
startstimes = [val/(sfreq) for val in starts]
endstimes = [val/(sfreq) for val in ends]

#Print and check
print(startstimes)
print(endstimes)

[10.5, 13.0, 15.5, 18.0, 20.5, 23.0, 25.5, 28.0, 30.5, 33.0, 35.5, 38.0, 40.5, 43.0, 45.5, 48.0, 50.5, 53.0, 55.5, 58.0, 60.5, 63.0, 65.5, 68.0, 100.51171875, 103.01171875, 105.51171875, 108.01171875, 110.51171875, 113.01171875, 115.51171875, 118.01171875, 120.51171875, 123.01171875, 125.51171875, 128.01171875, 130.51171875, 133.01171875, 135.51171875, 138.01171875, 140.51171875, 143.01171875, 145.51171875, 148.01171875, 150.51171875, 153.01171875, 155.51171875, 158.01171875, 190.5234375, 193.0234375, 195.5234375, 198.0234375, 200.5234375, 203.0234375, 205.5234375, 208.0234375, 210.5234375, 213.0234375, 215.5234375, 218.0234375, 220.5234375, 223.0234375, 225.5234375, 228.0234375, 230.5234375, 233.0234375, 235.5234375, 238.0234375, 240.5234375, 243.0234375, 245.5234375, 248.0234375]
[15.50390625, 18.00390625, 20.50390625, 23.00390625, 25.50390625, 28.00390625, 30.50390625, 33.00390625, 35.50390625, 38.00390625, 40.50390625, 43.00390625, 45.50390625, 48.00390625, 50.50390625, 53.00390625

In [20]:
epochsRest = []
labelsRest = []
for start,end in zip(starts,ends):
  epoch = dfRest.loc[start:end,eeg_channels_of_interest].values.T
  epochsRest.append(epoch.copy())
  labelsRest.append(0)
epochsRest = np.stack(epochsRest,axis=0)
labelsRest = np.array(labelsRest)

epochsRest.shape, labelsRest.shape

((72, 61, 1282), (72,))

#### **3. ¿How many epochs for -Lateral Grasp- are in the data and which are those epochs?**

In [21]:
### Analysis
# Filter by TypeRun equal to Rest
dfLG = dfFiltered[dfFiltered['TypeRun']=='AttemptedMovement']
dfLG.reset_index(inplace=True)
#Signal rest
signalLG = dfLG['LateralGraspClassCue'].values.tolist()
#Retrieve indexes where signal equals 1
indexesOnes = [i for i,val in enumerate(signalLG) if val == 1]
timesOnes = [val/(sfreq) for val in indexesOnes]
consecutiveOnes = [timesOnes[i+1]-timesOnes[i] for i,val in enumerate(timesOnes) if i+1 < len(timesOnes)]

### Report
print('Report on Lateral Grasp:')
print('Total samples = %d'%(len(signalLG)))
print('Sampling frequency (Hz) = %d'%(sfreq))
print('Signal duration (s) = %.2f'%(len(signalLG)/sfreq))
print('Signal duration (min) = %.2f'%(len(signalLG)/(60.0*sfreq)))
print('Total lateral grasp events = '+str(len(indexesOnes)))
print('Indexes (sample) = '+str(indexesOnes))
print('Times (s) = '+str(timesOnes))
print('Duration between consecutive ones (s) = '+str(consecutiveOnes))

Report on Lateral Grasp:
Total samples = 691044
Sampling frequency (Hz) = 256
Signal duration (s) = 2699.39
Signal duration (min) = 44.99
Total lateral grasp events = 72
Indexes (sample) = [8943, 17585, 24019, 29582, 32903, 46994, 61145, 64521, 80600, 82341, 87409, 93206, 110498, 126312, 128047, 142325, 166138, 167711, 171424, 183488, 186837, 217073, 219069, 222632, 231771, 244712, 248117, 258926, 264198, 278187, 283528, 287238, 316986, 320389, 335009, 338405, 345414, 347188, 349056, 356141, 401282, 408593, 413920, 415691, 428386, 433336, 450390, 453549, 467353, 475933, 477591, 501281, 503030, 511851, 523015, 530436, 539303, 555876, 566717, 574106, 577448, 581319, 589753, 602789, 620220, 623896, 631110, 635028, 640479, 652734, 680444, 682293]
Times (s) = [34.93359375, 68.69140625, 93.82421875, 115.5546875, 128.52734375, 183.5703125, 238.84765625, 252.03515625, 314.84375, 321.64453125, 341.44140625, 364.0859375, 431.6328125, 493.40625, 500.18359375, 555.95703125, 648.9765625, 655.121093

In [22]:
#Estimating start and end for every epoch
totalSamples = round(1 + ((tmax - tmin)*sfreq))
starts = []
ends = []
startstimes = []
endstimes = []

for starIndexOnes in indexesOnes:
  start = starIndexOnes + sfreq*tmin
  starts.append(start)
  ends.append(start+totalSamples)

#Transforming from sample to time
startstimes = [val/(sfreq) for val in starts]
endstimes = [val/(sfreq) for val in ends]

#Print and check
print(startstimes)
print(endstimes)

[32.93359375, 66.69140625, 91.82421875, 113.5546875, 126.52734375, 181.5703125, 236.84765625, 250.03515625, 312.84375, 319.64453125, 339.44140625, 362.0859375, 429.6328125, 491.40625, 498.18359375, 553.95703125, 646.9765625, 653.12109375, 667.625, 714.75, 727.83203125, 845.94140625, 853.73828125, 867.65625, 903.35546875, 953.90625, 967.20703125, 1009.4296875, 1030.0234375, 1084.66796875, 1105.53125, 1120.0234375, 1236.2265625, 1249.51953125, 1306.62890625, 1319.89453125, 1347.2734375, 1354.203125, 1361.5, 1389.17578125, 1565.5078125, 1594.06640625, 1614.875, 1621.79296875, 1671.3828125, 1690.71875, 1757.3359375, 1769.67578125, 1823.59765625, 1857.11328125, 1863.58984375, 1956.12890625, 1962.9609375, 1997.41796875, 2041.02734375, 2070.015625, 2104.65234375, 2169.390625, 2211.73828125, 2240.6015625, 2253.65625, 2268.77734375, 2301.72265625, 2352.64453125, 2420.734375, 2435.09375, 2463.2734375, 2478.578125, 2499.87109375, 2547.7421875, 2655.984375, 2663.20703125]
[37.9375, 71.6953125, 96.

In [23]:
epochsLG = []
labelsLG = []
for start,end in zip(starts,ends):
  epoch = dfLG.loc[start:end,eeg_channels_of_interest].values.T
  epochsLG.append(epoch.copy())
  labelsLG.append(1)
epochsLG = np.stack(epochsLG,axis=0)
labelsLG = np.array(labelsLG)

epochsLG.shape, labelsLG.shape

((72, 61, 1282), (72,))

#### **4. ¿How many epochs for -Palmar Grasp- are in the data and which are those epochs?**

In [24]:
### Analysis
# Filter by TypeRun equal to Rest
dfPG = dfFiltered[dfFiltered['TypeRun']=='AttemptedMovement']
dfPG.reset_index(inplace=True)
#Signal rest
signalPG = dfPG['PalmarGraspClassCue'].values.tolist()
#Retrieve indexes where signal equals 1
indexesOnes = [i for i,val in enumerate(signalPG) if val == 1]
timesOnes = [val/(sfreq) for val in indexesOnes]
consecutiveOnes = [timesOnes[i+1]-timesOnes[i] for i,val in enumerate(timesOnes) if i+1 < len(timesOnes)]

### Report
print('Report on Palmar Grasp:')
print('Total samples = %d'%(len(signalPG)))
print('Sampling frequency (Hz) = %d'%(sfreq))
print('Signal duration (s) = %.2f'%(len(signalPG)/sfreq))
print('Signal duration (min) = %.2f'%(len(signalPG)/(60.0*sfreq)))
print('Total palmar grasp events = '+str(len(indexesOnes)))
print('Indexes (sample) = '+str(indexesOnes))
print('Times (s) = '+str(timesOnes))
print('Duration between consecutive ones (s) = '+str(consecutiveOnes))

Report on Palmar Grasp:
Total samples = 691044
Sampling frequency (Hz) = 256
Signal duration (s) = 2699.39
Signal duration (min) = 44.99
Total palmar grasp events = 72
Indexes (sample) = [12348, 14235, 27539, 34798, 38215, 57730, 59516, 67881, 78581, 84314, 89274, 94755, 99695, 103158, 131762, 148100, 157071, 169570, 176737, 180043, 188644, 201256, 202986, 215257, 235335, 241319, 250064, 253413, 262245, 267448, 288830, 293662, 309221, 314984, 325424, 336807, 341870, 363078, 364909, 366953, 384610, 386392, 392179, 397461, 399408, 417314, 430162, 439103, 470766, 482849, 486695, 494129, 497341, 499340, 506444, 508236, 541263, 552146, 557493, 572224, 586069, 587947, 596857, 608283, 616990, 622069, 633053, 654408, 656011, 665559, 673363, 687156]
Times (s) = [48.234375, 55.60546875, 107.57421875, 135.9296875, 149.27734375, 225.5078125, 232.484375, 265.16015625, 306.95703125, 329.3515625, 348.7265625, 370.13671875, 389.43359375, 402.9609375, 514.6953125, 578.515625, 613.55859375, 662.3828125,

In [25]:
#Estimating start and end for every epoch
totalSamples = round(1 + ((tmax - tmin)*sfreq))
starts = []
ends = []
startstimes = []
endstimes = []

for starIndexOnes in indexesOnes:
  start = starIndexOnes + sfreq*tmin
  starts.append(start)
  ends.append(start+totalSamples)

#Transforming from sample to time
startstimes = [val/(sfreq) for val in starts]
endstimes = [val/(sfreq) for val in ends]

#Print and check
print(startstimes)
print(endstimes)

[46.234375, 53.60546875, 105.57421875, 133.9296875, 147.27734375, 223.5078125, 230.484375, 263.16015625, 304.95703125, 327.3515625, 346.7265625, 368.13671875, 387.43359375, 400.9609375, 512.6953125, 576.515625, 611.55859375, 660.3828125, 688.37890625, 701.29296875, 734.890625, 784.15625, 790.9140625, 838.84765625, 917.27734375, 940.65234375, 974.8125, 987.89453125, 1022.39453125, 1042.71875, 1126.2421875, 1145.1171875, 1205.89453125, 1228.40625, 1269.1875, 1313.65234375, 1333.4296875, 1416.2734375, 1423.42578125, 1431.41015625, 1500.3828125, 1507.34375, 1529.94921875, 1550.58203125, 1558.1875, 1628.1328125, 1678.3203125, 1713.24609375, 1836.9296875, 1884.12890625, 1899.15234375, 1928.19140625, 1940.73828125, 1948.546875, 1976.296875, 1983.296875, 2112.30859375, 2154.8203125, 2175.70703125, 2233.25, 2287.33203125, 2294.66796875, 2329.47265625, 2374.10546875, 2408.1171875, 2427.95703125, 2470.86328125, 2554.28125, 2560.54296875, 2597.83984375, 2628.32421875, 2682.203125]
[51.23828125, 58

In [26]:
epochsPG = []
labelsPG = []
for start,end in zip(starts,ends):
  epoch = dfPG.loc[start:end,eeg_channels_of_interest].values.T
  epochsPG.append(epoch.copy())
  labelsPG.append(2)
epochsPG = np.stack(epochsPG,axis=0)
labelsPG = np.array(labelsPG)

epochsPG.shape, labelsPG.shape

((72, 61, 1282), (72,))

#### 5. ¿How discriminable, on the basis of STM, are rest epochs from lateral grasp?

In [27]:
X = np.concatenate([epochsRest,epochsLG,epochsPG], axis=0)
y = np.concatenate([labelsRest,labelsLG,labelsPG])
X.shape, y.shape, y

((216, 61, 1282),
 (216,),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]))

In [28]:
clf_multi = OneVsRestClassifier(STM(C1=1.0, C2=1.0, maxIter=30, tolSTM=1e-4, penalty = 'l2', dual = True, tol=1e-4,loss = 'squared_hinge', maxIterSVM=100000, n_jobs=1))

In [29]:
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(X)

In [30]:
scores = cross_val_score(clf_multi, X, y, cv=cv, n_jobs=1)
print(scores)

1 / 30
deltau, deltav 7.810249675906773 35.805027579934894
2 / 30
deltau, deltav 0.01297833133861887 0.6360316107257434
3 / 30
deltau, deltav 0.00021012861667320788 0.11294142713023664
4 / 30


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



deltau, deltav 5.648041754445429e-05 0.00865049395499
5 / 30


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



deltau, deltav 5.723923082406016e-05 0.006226312543296199
6 / 30


...

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



deltau, deltav 0.0006917398678343957 0.025997362838397042
29 / 30


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



deltau, deltav 0.0006717823056297902 0.025136202343933236
30 / 30
deltau, deltav 0.000672263380125143 0.025190899401268525
[0.52272727 0.5        0.65909091 0.56818182 0.56818182 0.56818182
 0.40909091 0.52272727 0.59090909 0.45454545]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



In [31]:
for i,j in cv_split:
  print(len(i),len(j))

172 44
172 44
172 44
172 44
172 44
172 44
172 44
172 44
172 44
172 44


In [32]:
from scipy.stats import binom

In [33]:
alpha = 0.05
c = 3
n = 44
x = (100/n)*binom.ppf(1-alpha, n, 1/c)
print(x)

45.45454545454546
